### Personal Configuration

In [1]:
# Used to reload any self-made packages
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
from config import nomics_key

import sys
sys.path.append("../nomics-api")

from nomics import Nomics
nomics = Nomics(nomics_key)

# Basic Momentum Trading Strategy

In [3]:
import pandas as pd
import numpy as np

## Import Data
For the sake of this analysis, I'm only going to look at crypto markets on Coinbase Pro (GDAX) where the quote currency is USD.

In [5]:
# Get all markets on the GDAX exchange where the quote price is USD
usd_markets = pd.DataFrame(nomics.markets(exchange = 'gdax', quote = ['USD']))
usd_markets

base exchange   market quote
0  BCH     gdax  BCH-USD   USD
1  ZRX     gdax  ZRX-USD   USD
2  BTC     gdax  BTC-USD   USD
3  ETH     gdax  ETH-USD   USD
4  LTC     gdax  LTC-USD   USD
5  ETC     gdax  ETC-USD   USD

In [6]:
prices = pd.DataFrame()
for base in usd_markets['base']:
    base_prices = pd.DataFrame(nomics.candles(interval = '1d', base = base, start = '2018-01-01', 
                                              exchange = 'gdax', quote = 'USD')).set_index('timestamp')[['close']].astype(float)
    base_prices = base_prices.rename(columns = {
        'close' : base
    })
    prices = prices.join(base_prices, how = 'outer')
    
# Convert to iso format
prices.index = [pd.to_datetime(x) for x in prices.index]
prices.index.name = 'date'

In [7]:
prices.head()

BCH  ZRX       BTC     ETH     LTC  ETC
date                                                   
2018-01-01  2326.86  NaN  13480.01  759.03  225.22  NaN
2018-01-02  2558.58  NaN  14781.51  865.00  253.31  NaN
2018-01-03  2548.65  NaN  15098.14  938.31  244.99  NaN
2018-01-04  2362.74  NaN  15144.99  949.00  238.91  NaN
2018-01-05  2398.00  NaN  16960.01  969.27  246.23  NaN

## Basic Momentum Trading Strategy Analysis

Let's see what the returns of a Coinbase Pro portfolio would look like if we were to use a basic momentum based trading strategy. This strategy says that if the coin had positive returns during the previous period, then we should add it to the portfolio. We'll later analyze and compare the returns on this strategy when rebalancing daily, weekly, and monthly. 

In [8]:
# Calculate log returns
log_returns = np.log(prices) - np.log(prices.shift(1))

# Calculate the previous and future period returns
prev_period_returns = log_returns.shift(1)
future_period_returns = log_returns.shift(-1)

In [9]:
print("Previous Period Returns")
print(prev_period_returns.head())
print()
print("Future Period Returns")
print(future_period_returns.tail())

Previous Period Returns
                 BCH  ZRX       BTC       ETH       LTC  ETC
date                                                        
2018-01-01       NaN  NaN       NaN       NaN       NaN  NaN
2018-01-02       NaN  NaN       NaN       NaN       NaN  NaN
2018-01-03  0.094933  NaN  0.092169  0.130688  0.117536  NaN
2018-01-04 -0.003889  NaN  0.021194  0.081351 -0.033397  NaN
2018-01-05 -0.075742  NaN  0.003098  0.011328 -0.025130  NaN

Future Period Returns
                 BCH       ZRX       BTC       ETH       LTC       ETC
date                                                                  
2019-01-17 -0.023549 -0.046980 -0.009050 -0.024952 -0.021094 -0.008949
2019-01-18  0.014025  0.025187  0.020521  0.029998  0.039891  0.002245
2019-01-19 -0.053372 -0.052585 -0.039218 -0.047723 -0.051261 -0.015820
2019-01-20 -0.009317 -0.002803       NaN -0.020563  0.002610  0.002275
2019-01-21       NaN       NaN       NaN       NaN       NaN       NaN


In [10]:
# If the previous period had a positive return, then it's added to the portfolio
# When an asset is added to the portfolio, it's return is designated in the future_period_returns df

portfolio = (prev_period_returns > 0) * future_period_returns
portfolio.head()

BCH  ZRX       BTC       ETH      LTC  ETC
date                                                       
2018-01-01  0.000000  NaN  0.000000  0.000000  0.00000  NaN
2018-01-02 -0.000000  NaN  0.000000  0.000000 -0.00000  NaN
2018-01-03 -0.075742  NaN  0.003098  0.011328 -0.02513  NaN
2018-01-04  0.000000  NaN  0.113188  0.021134  0.00000  NaN
2018-01-05  0.000000  NaN  0.008161  0.037870  0.00000  NaN

In [11]:
# Divide the total return for that period by the number of coins in the portfolio
# We're assuming each coin was evenly weighted in the portfolio
# Fill any empty values with 0 (no return for that period)

portfolio_returns = (portfolio.sum(axis = 1) / (prev_period_returns > 0).sum(axis = 1)).fillna(0)
portfolio_returns.head()

date
2018-01-01    0.000000
2018-01-02    0.000000
2018-01-03   -0.021611
2018-01-04    0.067161
2018-01-05    0.023016
dtype: float64

In [12]:
#Let's assume we invested $100 on January 1, 2018. Where would we be now?
initial_value = 100
value = initial_value
portfolio_values = []
for date in portfolio_returns.index:
    value += portfolio_returns.loc[date] * value
    portfolio_values.append(value)

In [14]:
output = portfolio_returns.to_frame("Returns")
output["Portfolio Value"] = portfolio_values
output.head()

Returns  Portfolio Value
date                                 
2018-01-01  0.000000       100.000000
2018-01-02  0.000000       100.000000
2018-01-03 -0.021611        97.838858
2018-01-04  0.067161       104.409855
2018-01-05  0.023016       106.812902

In [15]:
max_return_date = output.idxmax()['Returns']
max_value_date = output.idxmax()['Portfolio Value']
end_value = output.iloc[-1]['Portfolio Value']
overall_return = (end_value - initial_value) / initial_value
# Win rate is the percentage of period with positive returns
win_rate = sum(output['Returns'] > 0) / len(output)

print("The overall return of this strategy is {}; the ending value of the portfolio is {}".format(round(overall_return,4), round(end_value, 2)))
print("The maximum period over period return of {} happened on {}".format(round(output.loc[max_return_date, 'Returns'], 4), max_return_date))
print("The maximum portfolio value of {} occured on {}".format(round(output.loc[max_value_date, 'Portfolio Value'], 2), max_value_date))
print("The win rate over this period is {}".format(round(win_rate, 4)))

The overall return of this strategy is -0.5868; the ending value of the portfolio is 41.32
The maximum period over period return of 0.1634 happened on 2018-12-19 00:00:00
The maximum portfolio value of 115.85 occured on 2018-01-08 00:00:00
The win rate over this period is 0.2824


16% in one day sounds impressive, but the overall loss of 58% is quite depressive. Let's see if rebalancing weekly or monthly can show an improvement. First, let's create a function to group together all of these calculations

In [16]:
def test_strategy(resample_freq):
    resampled_prices = prices.resample(resample_freq).last()

    # Calculate log returns
    log_returns = np.log(resampled_prices) - np.log(resampled_prices.shift(1))

    # Calculate the previous and future period returns
    prev_period_returns = log_returns.shift(1).replace(np.inf, np.nan).replace(-np.inf, np.nan)
    future_period_returns = log_returns.shift(-1).replace(np.inf, np.nan).replace(-np.inf, np.nan)

    # If the previous period had a positive return, then it's added to the portfolio
    # When an asset is added to the portfolio, it's return is designated in the future_period_returns df
    portfolio = (prev_period_returns > 0) * future_period_returns

    # Divide the total return for that period by the number of coins in the portfolio
    # We're assuming each coin was evenly weighted in the portfolio
    # Fill any empty values with 0 (no return for that period)

    portfolio_returns = (portfolio.sum(axis = 1) / (prev_period_returns > 0).sum(axis = 1)).fillna(0)

    #Let's assume we invested $100 on January 1, 2018. Where would we be now?
    initial_value = 100
    value = initial_value
    portfolio_values = []
    for date in portfolio_returns.index:
        value += portfolio_returns.loc[date] * value
        portfolio_values.append(value)

    output = portfolio_returns.to_frame("Returns")
    output["Portfolio Value"] = portfolio_values

    max_return_date = output.idxmax()['Returns']
    max_value_date = output.idxmax()['Portfolio Value']
    end_value = output.iloc[-1]['Portfolio Value']
    overall_return = (end_value - initial_value) / initial_value
    # Win rate is the percentage of period with positive returns
    win_rate = sum(output['Returns'] > 0) / len(output)

    print("The overall return of this strategy is {}; the ending value of the portfolio is {}".format(round(overall_return,4), round(end_value, 2)))
    print("The maximum period over period return of {} happened on {}".format(round(output.loc[max_return_date, 'Returns'], 4), max_return_date.date()))
    print("The maximum portfolio value of {} occured on {}".format(round(output.loc[max_value_date, 'Portfolio Value'], 2), max_value_date.date()))
    print("The win rate over this period is {}".format(round(win_rate, 4)))
    
    return output

In [17]:
print("Daily Rebalance")
daily_rebalance_output = test_strategy('1d')

Daily Rebalance
The overall return of this strategy is -0.5306; the ending value of the portfolio is 46.94
The maximum period over period return of 0.1634 happened on 2018-12-19
The maximum portfolio value of 115.85 occured on 2018-01-08
The win rate over this period is 0.2824


In [18]:
print("Weekly Rebalance")
weekly_rebalance_output = test_strategy('W')

Weekly Rebalance
The overall return of this strategy is -0.7424; the ending value of the portfolio is 25.76
The maximum period over period return of 0.1838 happened on 2018-04-15
The maximum portfolio value of 106.0 occured on 2018-01-21
The win rate over this period is 0.2143


In [19]:
print("Monthly Rebalance")
monthly_rebalance_output = test_strategy('M')

Monthly Rebalance
The overall return of this strategy is -0.1105; the ending value of the portfolio is 88.95
The maximum period over period return of 0.2661 happened on 2018-03-31
The maximum portfolio value of 126.61 occured on 2018-03-31
The win rate over this period is 0.0769


Here's something that's interesting. The strategy with the greatest ending portfolio value has the lowest win rate. 🤔 . Well, just for kicks, let's see what a passive (buy and hold) Coinbase Pro portfolio would look like. 

In [20]:
# Using the second to last day because as of right now, nomics isn't returning a price for BTC. Will review later. 
round(((prices.iloc[-2] - prices.iloc[0]) / prices.iloc[0]).sum() / 4, 4)

-0.8486

Well, although these momentum strategies didn't yield positive returns, they still outperformed a buy and hold strategy. However, given the extreme downturn of 2018's market, I'm sure beating the marketing wasn't hard to do. Regardless, I'll have to look around for other methods to use to trade in the crypto market; perhaps one that kicks into defensive mode when the market isn't doing too well. 

In [21]:
# To Do: play around with different return thresholds to see if results improve. 
# E.g. Only add a coin to the portfolio if the previous period's return was over 5%